In [2]:
from adtk.aggregator import OrAggregator
from adtk.data import validate_series
from adtk.detector import QuantileAD, InterQuartileRangeAD, GeneralizedESDTestAD, PersistAD
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import random as rdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, hamming_loss, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from tqdm.notebook import tqdm

In [3]:
cd ../

/home/guillaume/recimpute


In [4]:
%load_ext autoreload
%autoreload 2
from Clustering.AbstractClustering import AbstractClustering
from Clustering.ShapeBasedClustering import ShapeBasedClustering
from Datasets.Dataset import Dataset
from Datasets.TrainingSet import TrainingSet
from Labeling.ImputationTechniques.ImputeBenchLabeler import ImputeBenchLabeler
from Labeling.ImputationTechniques.KiviatRulesLabeler import KiviatRulesLabeler
from Labeling import AbstractLabeler
from FeaturesExtraction.TSFreshFeaturesExtractor import TSFreshFeaturesExtractor
from FeaturesExtraction.KiviatFeaturesExtractor import KiviatFeaturesExtractor
from Utils.Utils import Utils

In [ ]:
if not os.path.exists('Experiments/results'):
    os.makedirs('Experiments/results')

In [5]:
len(Dataset.CONF['USE_LIST'])

107

In [6]:
CLUSTERER = ShapeBasedClustering()
DATASETS = Dataset.instantiate_from_dir(CLUSTERER)
IB_LABELER = ImputeBenchLabeler.get_instance()
IB_LABELER_PROPERTIES = IB_LABELER.get_default_properties()

In [7]:
def get_dataset_by_name(datasets, name):
    for dataset in datasets:
        if dataset.name == name:
            return dataset
    raise Exception('Data set not found: %s.' % name)

## Load

In [8]:
def __init_test_set(datasets, strategy, test_size_percentage):
    test_set = []
    if strategy == 'one_cluster_every_two_dataset':
        # reserve one cluster every two data sets for the test set
        for dataset in datasets[::2]:
            for cid in dataset.cids:
                test_set.append(cid)
                break
        return 'clusters', test_set
    elif strategy == 'clusters_percentage':
        # reserve a fixed percentage of randomly selected clusters for the test set
        all_cids = list(chain.from_iterable([ds.cids for ds in datasets]))
        nb_test_clusters = int(np.ceil(len(all_cids) * test_size_percentage))
        test_set = rdm.sample(all_cids, nb_test_clusters)
        return 'clusters', test_set
    elif strategy == 'ts_percentage':
        all_ts_ids = list(range(0, sum(ds.nb_timeseries for ds in datasets)))
        nb_test_sequences = int(np.ceil(len(all_ts_ids) * test_size_percentage))
        test_set = rdm.sample(all_ts_ids, nb_test_sequences)
        return 'timeseries', test_set

    else:
        raise Exception('Test set reservation strategy not implemented: ', strategy)

In [9]:
def custom_load(datasets, data_to_load, clusterer, labeler, labeler_properties, test_set_level, test_set_ids):
    all_complete_datasets = []
        
    for dataset in datasets:

        # load labels - dataset_labels: df w/ 2 cols: Time Series ID and Label
        dataset_labels, labels_set = dataset.load_labels(labeler, labeler_properties)
        dataset_labels.set_index('Time Series ID', inplace=True)
        
        # load cassignment if the column is not there already
        dataset_cassignment = dataset.load_cassignment(clusterer)
        dataset_cassignment.set_index('Time Series ID', inplace=True)
        
        # load time series
        dataset_series = dataset.load_timeseries(transpose=True)
        dataset_series.columns = range(dataset_series.shape[1])
        
        to_concat = [dataset_labels, dataset_cassignment, dataset_series]
        # concat data set's labels, series and cassignment
        complete_dataset = pd.concat(to_concat, axis=1)
        
        complete_dataset['Data Set Name'] = dataset.name
        
        all_complete_datasets.append(complete_dataset) # this list contains train, val & test sets
        
    # merge the complete data sets (each row is a time serie's info)
    all_complete_datasets_df = pd.concat(all_complete_datasets, axis=0)
    
    # create new time series ID (tid must be unique across ALL data sets)
    all_complete_datasets_df.index = list(range(0, all_complete_datasets_df.shape[0]))
    all_complete_datasets_df.index.name = 'New Time Series ID'

    df_to_return = None
    if data_to_load != 'all':
        # only keep either the test set or both the training and validation sets
        if test_set_level == 'clusters':
            mask = all_complete_datasets_df['Cluster ID'].isin(test_set_ids) # series that are in the test set
        elif test_set_level == 'datasets':
            mask = all_complete_datasets_df['Data Set Name'].isin(test_set_ids) # series that are in the test set
        elif test_set_level == 'timeseries':
            mask = all_complete_datasets_df.index.isin(test_set_ids) # series that are in the test set
        mask = mask if data_to_load == 'test' else ~mask
        df_to_return = all_complete_datasets_df.loc[mask]
    else:
        # keep all data
        df_to_return = all_complete_datasets_df

    assert df_to_return is not None and labels_set is not None
    return df_to_return, labels_set

In [10]:
test_set_level, test_set_ids = __init_test_set(DATASETS, 'ts_percentage', .35)
all_data_info, labels_set = custom_load(
    DATASETS, 
    'all', 
    CLUSTERER, 
    IB_LABELER, IB_LABELER_PROPERTIES, 
    test_set_level, test_set_ids
)

In [11]:
all_data_info.head(3)

,Label,Cluster ID,0,1,2,3,4,5,6,7,...,6240,6241,6242,6243,6244,6245,6246,6247,6248,6249
New Time Series ID,,,,,,,,,,,,,,,,,,,,,
0,rosl,7935,-0.114538,-0.113988,-0.127480,-0.130141,-0.121514,-0.114630,-0.103433,-0.091134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,svt,7936,-0.124929,-0.113072,-0.102470,-0.091563,-0.071726,-0.058273,-0.039614,-0.028974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,svt,7936,-0.072244,-0.069892,-0.056518,-0.046070,-0.033572,-0.024464,-0.007726,0.013688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
all_data_info.shape

(70879, 6253)

## Predictions

In [13]:
def sample_cluster_2and10(all_timeseries, cluster, cluster_id):
    # sample 2 series from the given cluster and 10 from the ~cluster's dataset
    
    # select all the data set's sequences except the ones from the cluster we want to label
    available_timeseries = pd.concat([all_timeseries, cluster]).drop_duplicates(keep=False)
    # number of time series to feed to the benchmark in addition to the sequence we want to label and a second sequence from its cluster
    N_to_sample = min(ImputeBenchLabeler.CONF['NB_TS_FOR_BCHMK'], available_timeseries.shape[0])

    sequences_to_use = pd.concat([
        # 1st (and 2nd if cluster has at least 2 series) seq of cluster we want to label (benchmark will try to reconstruct this one)
        *[cluster.iloc[i].to_frame().T for i in range(0, int(cluster.shape[0] > 1)+1)],
        # N sequences from the same data set but not the same cluster
        available_timeseries.sample(N_to_sample, replace=False) 
    ])

    if sequences_to_use.shape[0] < 5:
        nb_seq_to_add = 5 - sequences_to_use.shape[0]
        if cluster.shape[0] - 2 >= nb_seq_to_add:
            sequences_to_use = pd.concat([
                sequences_to_use,
                *[cluster.iloc[-i-1].to_frame().T for i in range(nb_seq_to_add)]
            ])
        else: raise Exception('The data set does not have enough time series for the ImputeBench benchmark to run properly.')

    return sequences_to_use

In [14]:
def _get_irregularity_score(timeseries):
    all_anomalies = []
    for _, s_ in timeseries.iterrows(): # for each time series
        s = validate_series(s_)

        detectors = [
            QuantileAD(high=0.99, low=0.01),
            #    compares each time series value with historical quantiles

            InterQuartileRangeAD(c=1.5),
            #    based on simple historical statistics, based on interquartile range (IQR)

            GeneralizedESDTestAD(alpha=0.3),
            #    detects anomaly based on generalized extreme Studentized deviate (ESD) test

            PersistAD(c=3.0, side='positive'),
            #    compares each time series value with its previous values
        ]

        try:
            all_anomalies.extend([d.fit_detect(s) for d in detectors])
        except:
            all_anomalies.extend([np.zeros(len(s), dtype=bool) for d in detectors])

    all_anomalies = OrAggregator().aggregate(pd.DataFrame(np.array(all_anomalies).T, index=s.index))
    all_anomalies_bool = all_anomalies.astype('bool')
    anomalies_distribution = all_anomalies_bool.value_counts(normalize=False)
    anomalies_distribution = anomalies_distribution[True] if True in anomalies_distribution else 0
    anomalies_percentage = anomalies_distribution / len(s)

    return anomalies_percentage

In [15]:
def compute_features(dataset, cluster):
    date_range = {'start': '1974-1-1', 'periods': cluster.shape[1], 'freq': '10min'}
    cluster.columns = pd.date_range(**date_range)
    
    # feature 1: length
    length = dataset.timeseries_length

    # feature 2: irregularity score
    irregularity = _get_irregularity_score(cluster)

    # feature 3: pairwise correlation
    corr_matrix = np.array(cluster.T.corr())
    corr_upper_values = np.array(Utils.strictly_upper_triang_val(corr_matrix))
    correlation = corr_upper_values[~np.isnan(corr_upper_values)] # (remove NaNs)

    features_dict = {'Cluster ID': [None], 'Length': [length], 'Irregularity': [irregularity], 'Correlation': [correlation]}
    
    return features_dict

In [42]:
def _apply_rules(cluster_features):
    # apply thresholds
    thresholds = KiviatRulesLabeler.CONF['FEATURES_THRESHOLDS']
    binary_cluster_features = {
        'large_ts': int(cluster_features['Length'] >= thresholds['large_ts']),
        'irregular_ts': int(cluster_features['Irregularity'] > thresholds['irregular_ts']),
        'mixed_corr': int(cluster_features['STD Correlation'] > thresholds['mixed_corr']),
        'high_corr': int(cluster_features['Median Correlation'] > thresholds['high_corr']),
    }
    #print(binary_cluster_features)

    features_weights = KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']
    kiviat_values = KiviatRulesLabeler.KIVIAT_VALUES

    # compute a score for each algorithm
    scores = {}
    for algo in KiviatRulesLabeler.CONF['ALGORITHMS_LIST']:
        score = features_weights['efficient'] * kiviat_values[algo]['efficient']
        for name, value in binary_cluster_features.items():
            score += value * features_weights[name] * kiviat_values[algo][name]
        scores[algo] = score

    # label is the algorithms producing the highest score
    #return sorted(scores.items(), key=(lambda i: i[1]), reverse=True)
    return sorted(scores.keys(), key=(lambda key: scores[key]), reverse=True) # return sorted list of algos algorithm

In [56]:
KiviatRulesLabeler.CONF['FEATURES_THRESHOLDS']['large_ts'] = 750
KiviatRulesLabeler.CONF['FEATURES_THRESHOLDS']['irregular_ts'] = .10
KiviatRulesLabeler.CONF['FEATURES_THRESHOLDS']['mixed_corr'] = .20
KiviatRulesLabeler.CONF['FEATURES_THRESHOLDS']['high_corr'] = .70

In [57]:
KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']['efficient'] = 0.0 # just for runtime
KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']['large_ts'] = 0.0 # just for runtime
KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']['irregular_ts'] = 0.5
KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']['mixed_corr'] = 0.8
KiviatRulesLabeler.CONF['FEATURES_WEIGHTS']['high_corr'] = 0.8

In [59]:
nb_cids_to_use = len(all_data_info['Cluster ID'].unique())

In [100]:
def _get_labels(cid):
    ds_name = all_data_info.loc[all_data_info['Cluster ID'] == cid]['Data Set Name'].to_numpy()[0]
    all_timeseries = all_data_info.loc[all_data_info['Data Set Name'] == ds_name]
    cluster = all_data_info.loc[all_data_info['Cluster ID'] == cid]
    sample = sample_cluster_2and10(all_timeseries, cluster, cid)
    
    # compute Kiviat Features of this sample
    sample = sample.iloc[:, ~sample.columns.isin(['Data Set Name', 'Cluster ID', 'Label'])].dropna(axis=1)
    sample = sample.apply(pd.to_numeric, errors='coerce')
    features_dict = compute_features(
        get_dataset_by_name(DATASETS, ds_name), 
        sample
    )
    
    features_dict['Cluster ID'] = cid
    features = pd.DataFrame.from_dict(features_dict)
    features = features.set_index('Cluster ID')
    
    features['STD Correlation'] = features['Correlation'].apply(np.std)
    features['Median Correlation'] = features['Correlation'].apply(np.median)
    features = features.drop(columns=['Correlation'])
    
    features_per_cid[cid] = features # TODO delete
    
    # label the sample
    label_preds = _apply_rules(features)
    
    return cid, label_preds

In [ ]:
# do predictions
y_pred_per_cid = {}
features_per_cid = {}
with Pool() as pool:
    for res in tqdm(pool.imap_unordered(_get_labels, 
                                        rdm.sample(all_data_info['Cluster ID'].unique().tolist(), nb_cids_to_use)), 
                    total=nb_cids_to_use):
        cid, label = res
        y_pred_per_cid[cid] = label

In [48]:
pd.DataFrame.from_dict(y_pred_per_cid).to_csv('Experiments/results/kiviat_y_pred_per_cid.csv')

## Evaluation

In [61]:
def compute_at_k_scores(y_true, y_preds, scores, K=3):
    # rank at which each y_true is found in the sorted y_pred_proba (list of len = len(y_true))
    ranks = [y_preds_i.index(y_true_i) + 1 if y_true_i in y_preds_i else np.inf
             for y_true_i, y_preds_i in zip(y_true, y_preds)] 

    scores['Mean Reciprocal Rank'] = (1 / len(y_true)) * sum(1 / rank_i for rank_i in ranks)
    # average prec@K and recall@k
    prec_at_k = lambda K, rank_y_true: int(rank_y_true <= K) / K
    scores['Precision@3'] = sum(prec_at_k(K, rank_y_true) for rank_y_true in ranks) / len(y_true)
    recall_at_k = lambda K, rank_y_true: int(rank_y_true <= K) / 1
    scores['Recall@3'] = sum(recall_at_k(K, rank_y_true) for rank_y_true in ranks) / len(y_true)
    return scores

In [62]:
def clean_preds(y_true, y_pred, y_preds, labels_set):
    # remove test entries where true label isn't one that can be predicted by the Kiviat rules
    df = pd.DataFrame((y_true, y_pred))
    df = df[~df.isin(set(labels_set) - set(np.unique(y_pred)))].dropna(axis=1)
    y_true_ = df.iloc[0].to_numpy()
    y_pred_ = df.iloc[1].to_numpy()
    y_preds_ = [y_pred_i for i,y_pred_i in enumerate(y_preds) if i in df.columns.tolist()]
    assert y_pred_.shape[0] == y_true_.shape[0] == len(y_preds_)
    print(y_true_.shape)
    return y_true_, y_pred_, y_preds_

In [101]:
def print_scores(y_true, y_pred, y_preds, labels_set):
    average_strat = 'weighted'
    scores = {
        'Accuracy': accuracy_score(y_true, y_pred, normalize=True, sample_weight=None), 
        'F1-Score': f1_score(y_true=y_true, y_pred=y_pred, average=average_strat, zero_division=0),
        'Precision': precision_score(y_true=y_true, y_pred=y_pred, average=average_strat, zero_division=0).tolist(), 
        'Recall': recall_score(y_true=y_true, y_pred=y_pred, average=average_strat, zero_division=0).tolist(),
        'Hamming Loss': hamming_loss(y_true, y_pred),
    }
    scores = compute_at_k_scores(y_true, y_preds, scores, K=3)
    print(scores)
    return scores

In [ ]:
# evaluation
y_true, y_preds, y_pred = [], [], []
y_true_per_category = {category: [] 
                       for category in set(Dataset.CONF['CATEGORIES'].values())}
y_pred_per_category = {category: [] 
                       for category in set(Dataset.CONF['CATEGORIES'].values())}
y_preds_per_category = {category: [] 
                        for category in set(Dataset.CONF['CATEGORIES'].values())}
for row in all_data_info.itertuples(index=False):
    if row[all_data_info.columns.get_loc('Cluster ID')] in y_pred_per_cid:
        true_label = row[all_data_info.columns.get_loc('Label')]
        pred_labels = y_pred_per_cid[row[all_data_info.columns.get_loc('Cluster ID')]]

        y_true.append(true_label)
        y_preds.append(pred_labels)
        y_pred.append(pred_labels[0])
        
        ds_name = row[all_data_info.columns.get_loc('Data Set Name')]
        category = Dataset.CONF['CATEGORIES'][ds_name]
        y_true_per_category[category].append(true_label)
        y_preds_per_category[category].append(pred_labels)
        y_pred_per_category[category].append(pred_labels[0])
        
print('\n\n============================================================\n', '\033[1m Average metrics: \033[0m')
y_true_, y_pred_, y_preds_ = clean_preds(y_true, y_pred, y_preds, labels_set)
print_scores(y_true_, y_pred_, y_preds_, labels_set)

ConfusionMatrixDisplay.from_predictions(y_true=y_true_, y_pred=y_pred_, xticks_rotation=45)
plt.show()

#print('\n\n============================================================\n', '\033[1m Average metrics per category: \033[0m')
#for category in y_true_per_category.keys():
#    y_true_per_category[category], y_pred_per_category[category], y_preds_per_category[category] = clean_preds(
#        y_true_per_category[category], y_pred_per_category[category], y_preds_per_category[category], 
#        labels_set
#    )
#    print('\n~~ Category: %s ~~' % category)
#    print_scores(y_true_per_category[category], y_pred_per_category[category], y_preds_per_category[category], labels_set)
#
#    ConfusionMatrixDisplay.from_predictions(y_true=y_true_per_category[category], y_pred=y_pred_per_category[category], xticks_rotation=45)
#    plt.show()